In [44]:
import numpy as np
import cv2
import random
from PIL import Image
import depth_tools

In [45]:
def crop_center(pil_img, crop_width, crop_height):
#     print(pil_img.shape)
    img_width, img_height, _ = pil_img.shape
    return pil_img[(img_width - crop_width) // 2:
                    (img_height - crop_height) // 2,
                    (img_width + crop_width) // 2:
                    (img_height + crop_height) // 2, :]

In [79]:
def augment_zoom(img):
#     print(img.shape)
    h, w, s = img.shape
    zoom_range=[0.9, 1]
    random.seed(int(np.sum(img[:, :, 1])))
    scale = random.uniform(zoom_range[0], zoom_range[1])
    print(scale)
    resize_w, resize_h = int(w*scale), int(h*scale)
    if resize_w % 2 == 1:
        resize_w += 1
    if resize_h % 2 == 1:
        resize_h += 1
        
    x = cv2.resize(img, (resize_w, resize_h))

    x = x / scale
    print(resize_w, resize_h)
    if scale > 1:
        new_img = x[int((resize_h - h)/2): int((resize_h + h)/2),
                   int((resize_w - w)/2): int((resize_w + w)/2), :]
    else:
        new_img = np.zeros_like(img)
        new_img[int((h - resize_h)/2): int((h + resize_h)/2),
                int((w - resize_w)/2): int((w + resize_w)/2), :] = x
    print(new_img.shape)
    return new_img

In [88]:
data_dir = '../../data/input_200117/'
rec_img = cv2.imread(data_dir + 'rec/depth001.png', -1)
gt_img = cv2.imread(data_dir + 'gt/gt001.bmp', -1)
shading = cv2.imread(data_dir + 'shading/shading000.png', -1)
frame = cv2.imread(data_dir + 'frame/frame000.png', -1)
# shading = cv2.imread('../../../tmp_program/sample_1.png', -1)
# frame = cv2.imread('../../../tmp_program/sample_2.png', -1)

rec = depth_tools.unpack_png_to_float(rec_img)
gt = depth_tools.unpack_bmp_bgra_to_float(gt_img)
print(gt)

[[2.98023224e-08 2.98023224e-08 2.98023224e-08 ... 2.98023224e-08
  2.98023224e-08 2.98023224e-08]
 [2.98023224e-08 2.98023224e-08 2.98023224e-08 ... 2.98023224e-08
  2.98023224e-08 2.98023224e-08]
 [2.98023224e-08 2.98023224e-08 2.98023224e-08 ... 2.98023224e-08
  2.98023224e-08 2.98023224e-08]
 ...
 [2.98023224e-08 2.98023224e-08 2.98023224e-08 ... 2.98023224e-08
  2.98023224e-08 2.98023224e-08]
 [2.98023224e-08 2.98023224e-08 2.98023224e-08 ... 2.98023224e-08
  2.98023224e-08 2.98023224e-08]
 [2.98023224e-08 2.98023224e-08 2.98023224e-08 ... 2.98023224e-08
  2.98023224e-08 2.98023224e-08]]


In [89]:
img = np.dstack([gt, rec])
# img = np.dstack([frame, shading])

In [90]:
new_img = augment_zoom(img)

0.9323637338587499
1118 1118
(1200, 1200, 2)


In [91]:
new_gt = new_img[:, :, 0]
new_rec = new_img[:, :, 1]
# new_frame = new_img[:, :, 0:3]
# new_shading = new_img[:, :, 3:6]
# new_frame = new_img[:, :, 0:4]
# new_shading = new_img[:, :, 4:8]

In [92]:
new_gt_img = depth_tools.pack_float_to_bmp_bgra(new_gt)
new_rec_img = depth_tools.pack_float_to_bmp_bgra(new_rec)
print(new_gt_img.shape)
# new_gt_img = new_gt
# new_rec_img = new_rec

(1200, 1200, 4)


In [93]:
cv2.imwrite('new_gt1.bmp', new_gt_img)
cv2.imwrite('new_rec1.bmp', new_rec_img)
# cv2.imwrite('new_frame.png', new_frame)
# cv2.imwrite('new_shading.png', new_shading)

True

In [94]:
new_gt

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [95]:
cam_params = {
    'focal_length': 0.036640125,
    'pix_x': 1.25e-05,
    'pix_y': 1.2303973256411377e-05,
    'center_x': 801.895,
    'center_y': 602.872
}
xyz = depth_tools.convert_depth_to_coords(new_gt, cam_params)
depth_tools.dump_ply('new_gt1.ply', xyz.reshape(-1, 3).tolist())

In [96]:
15//2

7

In [97]:
15/2

7.5